In [31]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
from torchinfo import summary
import pandas as pd
import os
from pathlib import Path
import soundfile

soundfile


In [32]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

class CNNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(51136, 50)
        self.fc2 = nn.Linear(50, 2)


    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        #x = x.view(x.size(0), -1)
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        return F.log_softmax(x,dim=1)  


Using cpu device


In [33]:
def load_audio_file(path: str):

    dataset = []
    walker = sorted(str(p) for p in Path(path).glob(f'*.wav'))

    for i, file_path in enumerate(walker):
        path, filename = os.path.split(file_path)
        # Load audio
        waveform, sample_rate = torchaudio.load(file_path)
        dataset.append([waveform, sample_rate])

    return dataset

In [34]:
print(str(torchaudio.get_audio_backend()))
model = CNNet()
model.load_state_dict(torch.load('./data/model.pth',map_location=torch.device('cpu')))
model.eval()

speechcommand = load_audio_file('./testaudio')

testloader = torch.utils.data.DataLoader(speechcommand, batch_size=1,
                                            shuffle=True, num_workers=0)


soundfile


In [ ]:
spectrogram_tensor = None
directory = f'./data/testimage/order'
if(os.path.isdir(directory)):
    print("Data exists")
else:
    os.makedirs(directory, mode=0o777, exist_ok=True)
spectorgram_value = None
for data in testloader: 
    waveform = data[0]
    spectrogram_tensor = torchaudio.transforms.Spectrogram()(waveform)
    """
    spectrogram = torch.tensor(spectrogram_tensor[0].log2()[0,:,:].numpy())
    spectrogram += abs(spectrogram.min())
    spectrogram = torch.nn.functional.normalize(spectrogram, dim=0, p=2)
    spectrogram /= spectrogram.max()

    print(len(spectrogram), len(spectrogram[0]))
    print(spectrogram)
    """
    fig = plt.figure()
    plt.imsave(f'./data/testimage/order/test.png', spectrogram_tensor[0].log2()[0,:,:].numpy(), cmap='viridis')
    

In [ ]:
dataset = datasets.ImageFolder(
    root="./data/testimage/",
    transform=transforms.Compose([transforms.Resize((201,81)),
                                  transforms.ToTensor()])
)
print(dataset[0])

In [37]:
test_dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=15,
    num_workers=2,
    shuffle=True
)


In [38]:
classes = [
    "no",
    "yes"
]


In [40]:
model.eval()
for batch, (X,Y) in enumerate(test_dataloader):
    pred = model(X)
    predicted = classes[pred[0].argmax(0)]
    prediction = pred[0].softmax(0)
    print(prediction[pred[0].argmax(0)])
    if prediction[pred[0].argmax(0)] > 0.95:
        
        print(f'Predicted: "{predicted}"')
    else:
        print("no definite answer")

tensor(0.9017, grad_fn=<SelectBackward0>)
no definite answer
